## Terraforming Mars Cost-Benefit Analysis
<br>
Terraforming Mars is a competitive board game developed by Jacob Fryxelius and published by FryxGames in 2016. Players take on the role of corporations tasked with making Mars habitable through the raising of the oxygen level, temperature, and ocean coverage, as well as sundry other projects to progress humanity.

Currently sitting at #4 on BoardGameGeek's ranked board game list, Terraforming Mars has found broad popularity among strategy board gamers in large part due to its variety and complexity. With 208 unique cards and 12 distinct corporations, players are put into an endless assortment of scenarios. This incredible depth means that players must often intuit their next move rather than deduce it from values and probabilities.

In a bid to add more deduction back into the equation, this analysis aims to take every card in Terraforming Mars and boil it down to a single value in credits (the game's main resource) for simple value-comparison.

Let's get started.

In [7]:
import numpy as np
from statistics import median
import pandas as pd
import seaborn as sns
import csv
import ast

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


The base_cards database, populated from a hand-entered csv, follows a couple of conventions:
* When a resource is written in uppercase it refers to that resource's production, whereas when written in lowercase it refers to the resource iself.
* All nouns are written in the singular in order to simplify pandas referencing.

The all_games database is a collection of stats from over 12,000 individually logged games downloaded from Simeon Simeonov's excellent [Terraforming Mars website](https://ssimeonoff.github.io/). The base_games database contains only the matches from all_games which were played with the full base version of the game.

In [8]:
# create databases
base_cards = pd.read_csv('TM_base_project_list.csv')
all_games = pd.read_csv('games_list.csv')
base_games = all_games[(all_games.expansions == "['CORPORATE']") & (all_games.map == "THARSIS")]

display(base_cards.head(10))

,Title,Game,Type,Primary Cost,Additional Cost,Tags,Prerequisites,Victory Points,Immediate Benefit,Passive Benefit,Active cost,Active Benefit,Removed from Opponent
0,Mineral Deposit,corporate,event,5,NaN,event,NaN,NaN,5 steel,NaN,NaN,NaN,NaN
1,Fusion Power,base,automated,14,NaN,"science, power, building",2 power tag,NaN,3 ENERGY,NaN,NaN,NaN,NaN
2,Black Polar Dust,base,automated,15,2 CREDIT,NaN,NaN,NaN,"3 HEAT, 1 ocean",NaN,NaN,NaN,NaN
3,Ice Cap Melting,base,event,5,NaN,event,+2°C,NaN,1 ocean,NaN,NaN,NaN,NaN
4,Underground Detonation,base,active,6,NaN,building,NaN,NaN,NaN,NaN,10 credit,2 HEAT,NaN
5,Energy Saving,base,automated,15,NaN,power,NaN,NaN,1 ENERGY per city,NaN,NaN,NaN,NaN
6,Local Heat Trapping,base,event,1,5 heat,event,NaN,NaN,4 plant or 2 animal on other card,NaN,NaN,NaN,NaN
7,Energy Tapping,corporate,automated,3,NaN,power,NaN,-1,1 ENERGY,NaN,NaN,NaN,1 ENERGY
8,Designed Microorganisms,base,automated,16,NaN,"science, microbe",max -14°C,NaN,2 PLANT,NaN,NaN,NaN,NaN
9,Power Infrastructure,corporate,active,4,NaN,"power, building",NaN,NaN,NaN,NaN,x energy,x credit,NaN


First, an average number of generations for each player count must be derived from base_games. These will be used to determine the income which will be generated over the remainder of the game for a given card after the generation during which it is played. They'll also be used to split the game into four quarters to illustrate how a card's value changes over time.

In [9]:
# determine generation averages per player count
lastgen2p = int(round(base_games[base_games.players == 2].generations.mean()))
lastgen3p = int(round(base_games[base_games.players == 3].generations.mean()))
lastgen4p = int(round(base_games[base_games.players == 4].generations.mean()))
lastgen5p = int(round(base_games[base_games.players == 5].generations.mean()))

lastgens = {
            'lastgen2p': int(round(base_games[base_games.players == 2].generations.mean())),
            'lastgen3p': int(round(base_games[base_games.players == 3].generations.mean())),
            'lastgen4p': int(round(base_games[base_games.players == 4].generations.mean())),
            'lastgen5p': int(round(base_games[base_games.players == 5].generations.mean()))
           }

print(f'''
Average length of game in generations:
2p: {lastgen2p}
3p: {lastgen3p}
4p: {lastgen4p}
5p: {lastgen5p}
''')

# average game length split into quarters with the median generation taken from each quarter
# X players, quarter N, median generation
gen_quarters = {
               '2pq1gen':round(lastgen2p/4/2),
               '2pq2gen':round(lastgen2p/4/2 + lastgen2p/4),
               '2pq3gen':round(lastgen2p/4/2 + lastgen2p/4*2),
               '2pq4gen':round(lastgen2p/4/2 + lastgen2p/4*3),
               '3pq1gen':round(lastgen3p/4/2),
               '3pq2gen':round(lastgen3p/4/2 + lastgen3p/4),
               '3pq3gen':round(lastgen3p/4/2 + lastgen3p/4*2),
               '3pq4gen':round(lastgen3p/4/2 + lastgen3p/4*3),
               '4pq1gen':round(lastgen4p/4/2),
               '4pq2gen':round(lastgen4p/4/2 + lastgen4p/4),
               '4pq3gen':round(lastgen4p/4/2 + lastgen4p/4*2),
               '4pq4gen':round(lastgen4p/4/2 + lastgen4p/4*3),
               '5pq1gen':round(lastgen5p/4/2),
               '5pq2gen':round(lastgen5p/4/2 + lastgen5p/4),
               '5pq3gen':round(lastgen5p/4/2 + lastgen5p/4*2),
               '5pq4gen':round(lastgen5p/4/2 + lastgen5p/4*3)
              }
keys2pgen = ['2pq1gen', '2pq2gen', '2pq3gen', '2pq4gen']
keys3pgen = ['3pq1gen', '3pq2gen', '3pq3gen', '3pq4gen']
keys4pgen = ['4pq1gen', '4pq2gen', '4pq3gen', '4pq4gen']
keys5pgen = ['5pq1gen', '5pq2gen', '5pq3gen', '5pq4gen']

print(f'''
Generation quarters:
2p: {str([gen_quarters.get(key) for key in keys2pgen]).strip('[]')}
3p: {str([gen_quarters.get(key) for key in keys3pgen]).strip('[]')}
4p: {str([gen_quarters.get(key) for key in keys4pgen]).strip('[]')}
5p: {str([gen_quarters.get(key) for key in keys5pgen]).strip('[]')}
''')



Average length of game in generations:
2p: 13
3p: 11
4p: 10
5p: 10


Generation quarters:
2p: 2, 5, 8, 11
3p: 1, 4, 7, 10
4p: 1, 4, 6, 9
5p: 1, 4, 6, 9



Next, each resource in the game must be converted to a number of credits. The [standard projects](https://3.bp.blogspot.com/-fs7vQZGRFs4/XAjAYOSBspI/AAAAAAAAFSA/kInbyuaLoLc0PANwJeFDpJditT5Ims3_wCLcBGAs/s1600/terraforming-mars-standard-projects-1.jpg) are set as listed on the game board, and steel and titanium as listed on the [player boards](https://cf.geekdo-images.com/medium/img/qa7YwBE6pc-ZR0relFRrzUAlQXw=/fit-in/500x500/filters:no_upscale()/pic2891980.jpg).  

Plants and heat are set by dividing 8, the number of plants/heat needed to equal a temp/greenery (also illustrated on the player boards), into the credit values for the temp and greenery standard projects.  

For energy, it was necessary to borrow some information from one of the expansions: _Colonies_. In order to trade with a colony, players must spend either 9 credits, 3 titanium, or 3 energy. This suggests that 1 energy = 1 titanium = 3 credits.  

Drawing a card is valued at 4 credits, combining the price to buy a card (3 credits) with the 1 credit gained from discarding it using the "Sell patents" standard project.

In [11]:
# variable definitions

income = 0

# standard projects
temp = 14
ocean = 18
greenery = 23
ENERGY = 11
city = 25 + income

# resources
steel = 2
titanium = 3
plant = greenery / 8
heat = temp / 8
energy = 3
draw = 4

Victory points (VP) must also be converted to credits. To do this, the three terraforming standard projects were used. Since they confer a terraforming rating (TR) and not a victory point, the extra income from the TR must be subtracted from the cost of the project. This means that similar to the cards themselves, the value of a VP changes over time.

To come to a single value for a given generation and player count, the "points per credit" are determined for each of the three standard terraforming projects and then the average is taken. Note that greeneries are worth two points (one for the TR from raising the oxygen, another from the greenery tile itself) and so are calculated accordingly.

In [12]:
# define credits per victory point
def credits_per_vp(generation, player_count):
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    income = lastgen - generation
    temp_ppc = 1/(temp - income) # ppc = points per credit
    ocean_ppc = 1/(ocean - income)
    greenery_ppc = 2/(greenery - income)
    ppc = (temp_ppc + ocean_ppc + greenery_ppc)/4
    credits_per_vp = 1/ppc
    return round(credits_per_vp)

# numbers of credits per victory point for each player count and median generation
# X players, quarter N, credits per point
gen_credits_per_vp = {'2pq1cpp':credits_per_vp(2, 2),
                      '2pq2cpp':credits_per_vp(5, 2),
                      '2pq3cpp':credits_per_vp(8, 2),
                      '2pq4cpp':credits_per_vp(11, 2),
                      '3pq1cpp':credits_per_vp(1, 3),
                      '3pq2cpp':credits_per_vp(4, 3),
                      '3pq3cpp':credits_per_vp(7, 3),
                      '3pq4cpp':credits_per_vp(10, 3),
                      '4pq1cpp':credits_per_vp(1, 4),
                      '4pq2cpp':credits_per_vp(4, 4),
                      '4pq3cpp':credits_per_vp(6, 4),
                      '4pq4cpp':credits_per_vp(9, 4),
                      '5pq1cpp':credits_per_vp(1, 5),
                      '5pq2cpp':credits_per_vp(4, 5),
                      '5pq3cpp':credits_per_vp(6, 5),
                      '5pq4cpp':credits_per_vp(9, 5)
                     }

keys2pcpp = ['2pq1cpp', '2pq2cpp', '2pq3cpp', '2pq4cpp']
keys3pcpp = ['3pq1cpp', '3pq2cpp', '3pq3cpp', '3pq4cpp']
keys4pcpp = ['4pq1cpp', '4pq2cpp', '4pq3cpp', '4pq4cpp']
keys5pcpp = ['5pq1cpp', '5pq2cpp', '5pq3cpp', '5pq4cpp']

print(f'''
Generation credits per point per quarter:
   q1, q2, q3, q4
2p: {str([gen_credits_per_vp.get(key) for key in keys2pcpp]).strip('[]')}
3p: {str([gen_credits_per_vp.get(key) for key in keys3pcpp]).strip('[]')}
4p: {str([gen_credits_per_vp.get(key) for key in keys4pcpp]).strip('[]')}
5p: {str([gen_credits_per_vp.get(key) for key in keys5pcpp]).strip('[]')}
''')


Generation credits per point per quarter:
   q1, q2, q3, q4
2p: 6, 10, 13, 17
3p: 8, 11, 14, 18
4p: 9, 12, 14, 18
5p: 9, 12, 14, 18



This brings me to the point of actually calculating the card values. My plan is to write a main function for doing this, then running it for most of the cards to create a values database.

Beyond that I've set aside fifteen active cards that have a cost to use. Ironworks, for example, costs 4 energy to use and grants a steel and raises the oxygen by 1. Do I calculate cards such as these as if they would be used in every remaining generation? That's probably what I'll do, I just haven't decided yet.

Finally there is a separate group of 23 cards that I don't know how to handle. An example would be Advanded Alloys, which increases the value of steel and titanium by one credit each. I'll probably have to have a separate function for each of those. Hopefully I'll be able to even figure out how to put numbers to each of them.

Before I start writing the probably very long function to determine the value of most of the cards, I will try a few card-specific functions just to get some practice and ideas about how to write the big one.

In [14]:
# Food Factory test
def food_factory(generation, player_count):
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    income = lastgen - generation
    #determine quarter of game
    quarters_list = []
    for gen in gen_quarters:
        if gen[0] == str(player_count):
            quarters_list.append(gen_quarters[gen])
    absolute_difference_function = lambda list_value : abs(list_value - generation)
    for i in range(4):
        if quarters_list[i] == min(quarters_list, key=absolute_difference_function):
            quarter = i+1
    #determine credits per vp
    for cpp in gen_credits_per_vp:
        if cpp[0] == str(player_count) and cpp[3] == str(quarter):
            vp = gen_credits_per_vp[cpp]
    value = 0
    value -= 12 #card cost
    value -= round(plant*(income+1))
    value += round(4*income)
    value += vp
    display(f'The value of Food Factory in gen {str(generation)} with {str(player_count)} players is {str(value)} credits')
    
food_factory(1, 2)

'The value of Food Factory in gen 1 with 2 players is 5 credits'